In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import os
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from pandas.util import hash_pandas_object

In [2]:
data = pd.read_csv('../data/en.openfoodfacts.org.products.csv', sep = '\t')

/Users/Matt/anaconda3/envs/ada/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,24,25,26,28,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
header = data.columns
odf = pd.DataFrame()
for index in header:
    odf[index] =( [index , sum(data[index].notnull())/1023451*100  , data[index].value_counts().head(4).index.tolist() ])
odf=odf.T
odf =odf.drop(columns=0)
odf=odf.rename(columns={1: "%", 2: "exemple top-4"})

In [4]:
odfcut=odf[odf["%"]>=3]

In [5]:
odfcut.to_csv('odfcut.csv')


Categories each product according to the PNNS

In [6]:
pnns_category = {'Dairies','Composite','Fish Meat Eggs','Beverages','Fat Sauces','Fruits Vegetables','Starchy','Snacks'}

In [7]:
data.pnns_groups_1.value_counts()

unknown                    716232
Sugary snacks               57997
Milk and dairy products     43467
Fish Meat Eggs              42475
Cereals and potatoes        34187
Beverages                   29118
Fat and sauces              28114
Composite foods             25430
Fruits and vegetables       24419
Salty snacks                19825
sugary-snacks                3469
fruits-and-vegetables        2898
cereals-and-potatoes           43
salty-snacks                    5
Name: pnns_groups_1, dtype: int64

In [8]:
pnns1_category = {'unknown' : 'Unknown',
 'Sugary snacks' : 'Snacks', 
 'Milk and dairy products' : 'Dairies',
 'Composite foods' : 'Composite', 
 'Cereals and potatoes' : 'Starchy', 
 'Fish Meat Eggs' : 'Fish Meat Eggs',
 'Beverages' : 'Beverages',
 'Fat and sauces' : 'Fat Sauces',
 'Fruits and vegetables' : 'Fruits Vegetables',
 'Salty snacks' : 'Snacks',
 'fruits-and-vegetables' : 'Fruits Vegetables',
 'sugary-snacks' : 'Snacks',
 'cereals-and-potatoes' : 'Starchy',
 'salty-snacks' : 'Snacks'
}
data['Category_Food'] = data['pnns_groups_1'].replace(pnns1_category)

In [9]:
data.Category_Food.value_counts()

Unknown              716232
Snacks                81296
Dairies               43467
Fish Meat Eggs        42475
Starchy               34230
Beverages             29118
Fat Sauces            28114
Fruits Vegetables     27317
Composite             25430
Name: Category_Food, dtype: int64

More category from 'main_category_en' source

In [10]:
data['main_category_en'].value_counts().head(30)

Plant-based foods and beverages    86518
Snacks                             51656
Beverages                          43688
Dairies                            38129
Meats                              31497
Groceries                          23517
Meals                              17557
Desserts                            7435
Spreads                             7272
Seafood                             7198
Fruit juices                        6415
Canned foods                        4731
Frozen foods                        4138
Biscuits and cakes                  3412
Sweeteners                          2488
Dietary supplements                 1995
Breakfasts                          1861
Vinegars                            1764
Sandwiches                          1760
Fats                                1674
Farming products                    1631
Fish and meat and eggs              1604
Fruit-juices-and-nectars            1241
Baby foods                          1231
Pies            

In [11]:
maincategoryen_category = {'unknown' : 'Unknown',
'Plant-based foods and beverages' : 'Brevages',
'Snacks' : 'Snacks',
'Beverages' : 'Beverages',
'Dairies' : 'Dairies' ,
'Meats' : 'Fish Meat Eggs',
'Meals' : 'Composite',
'Desserts' : 'Snacks',
'Seafood' : 'Fish Meat Eggs',
'Fruit juices' : 'Brevages',
'Biscuits and cakes' : 'Snacks',
'Fats' : 'Fat Sauces', 
'Fish and meat and eggs' : 'Fish Meat Eggs',
'Fruit-juices-and-nectars' : 'Brevages'
                          }

data['Category_Food_frommaincategoryen'] = data['main_category_en'].replace(maincategoryen_category)

In [12]:
data['Category_Food_frommaincategoryen'] = np.where(data['Category_Food_frommaincategoryen'].isin(pnns_category)==False,'Unknown',data['Category_Food_frommaincategoryen'])

In [13]:
data['Category_Food']= np.where(data['Category_Food']=='Unknown', data['Category_Food_frommaincategoryen'], data['Category_Food'])
                                                   

In [14]:
data.pnns_groups_1.value_counts()

unknown                    716232
Sugary snacks               57997
Milk and dairy products     43467
Fish Meat Eggs              42475
Cereals and potatoes        34187
Beverages                   29118
Fat and sauces              28114
Composite foods             25430
Fruits and vegetables       24419
Salty snacks                19825
sugary-snacks                3469
fruits-and-vegetables        2898
cereals-and-potatoes           43
salty-snacks                    5
Name: pnns_groups_1, dtype: int64

In [15]:
data.Category_Food.value_counts()

Unknown              700489
Snacks                86055
Dairies               45034
Fish Meat Eggs        42484
Beverages             38526
Starchy               34230
Fat Sauces            28114
Fruits Vegetables     27317
Composite             25430
Name: Category_Food, dtype: int64

In [16]:
data[data['Category_Food']=='Unknown']['categories'].value_counts()

Compléments alimentaires                                                                                                             350
Protein bars                                                                                                                         328
Pancake mixes                                                                                                                        260
Licensed products                                                                                                                    258
Maple syrups                                                                                                                         235
                                                                                                                                    ... 
Non pasteurisé                                                                                                                         1
Iogurte de Beber                         

In [17]:
data.packaging.value_counts()[19:40]

barquette                      1141
Sachet,plastique               1050
carton,plastique                998
Sachet plastique                975
conserve                        957
pot                             944
Karton                          904
Bocal,Verre,Couvercle,Métal     874
Carton,Plastique                847
Sachet                          809
boîte                           739
carton,boîte,plastique          718
bouteille,plastique             716
Plastik                         703
Plástico                        687
sachet plastique                681
plastic                         660
Dose                            653
Stück                           641
Barquette,Plastique,Frais       581
bouteille,verre                 578
Name: packaging, dtype: int64

In [20]:
pack_mapping = dict.fromkeys(['plastic', 'plastique,frais','plastique', 'barquette,plastique', 'sachet,plastique',
                              'frais,plastique','carton,plastique','carton,plastique,surgele','flacon,plastique'
                 'sachet-plastique','carton,boite,plastique', 'boite,plastique', 'plastique,carton','sachet-plastique',
                'sachet,plastique,surgele','barquette,plastique,frais','pot,plastique','plastique,sachet',
                             'plastico','boite,carton,plastique','frais,plastique,carton','frais,carton,plastique',
                              'carton,sachet,plastique','kunststoff','barquette,film,plastique,sous-atmosphere-protectrice',
                             'carton,surgele,plastique','plastique,barquette','sachet,plastique,frais',
                             'barquette,plastique,sous-atmosphere-protectrice','frais,barquette,plastique,opercule,film-plastique,sous-atmosphere-protectrice',
                             'sachet,plastique,carton','film,plastique','sachet,plastique,sous-atmosphere-protectrice',
                             'bolsa-de-plastico,ultracongelado','barquette-plastique','plastique,sous-vide','plastic,bag',
                             'barquette,film,plastique','plastik','plastica','pot-plastique','plastic-bag','boite-plastique',
                             'frais,barquette,plastique','05-pp','pp','sachet,plastique,etui,carton','bolsa-de-plastico',
                             'boite,carton,sachet,plastique','pot,plastique,frais','botella,plastico','sachet,plastique,doypack',
                             'sachet,plastique,sachet','plastique,boite','plastique,surgele','sac-plastique'],'Plastic')

pack_mapping.update(dict.fromkeys(['bouteille,verre','carton','boite,carton','carton,surgele','karton','carton,boite','karton,kunststoff',
                                  'sachet,papier','papier','bouteille,plastique','conserve','bocal,verre','glas',
                                  'bouteille','conserve,metal','bocal,verre,couvercle,metal','bocal,verre,metal',
                                  'verre','verre,bouteille','pot,verre','bocal','verre,bocal',
                                  'flacon,plastique','bouteille-plastique','bottle','canned','pot-en-verre','bolsa,plastico',
                                   'bouteille-verre','bocal-en-verre','verre,bocal,metal','plastique,bouteille','bouteille-en-verre','canette','pot,verre,couvercle,metal',
                                  'glas,mehrwegpfand','carton,aluminium','boite,metal','botella-de-plastico','boite-carton',
                                  'plastic-bottle','sous-vide','plastic,bottle','can','pot-en-plastique','glass','caja,carton','bote-de-vidrio',
                                  'boite,conserve,metal','pet','becher','lata','bocal-verre','aluminium','caja-de-carton','bouteille,verre,capsule,metal',
                                  'bokaal,glas','konserve','paper','glass-bottle','bocal,verre,couvercle,metal,conserve','glass-jar',
                                  'glasflasche','flacon-verre','glas,flasche','kunststoff,karton','papier,aluminium','boite-de-conserve',
                                  'pot-verre','canette,metal','carton,brique','flacon,verre','caja,carton,lata,en-conserva','bocal-verre,couvercle-metal','glass,bottle',
                                  'conserve,boite,metal','carton,sachet','canette,aluminium','boite-en-carton','conserve,conserve'],'Recyclable'))

pack_mapping.update(dict.fromkeys(['frais','surgele','surgele,carton,plastique','brique,carton','tetra-pak','tetra-brik','tetrapak',
                                  'brique','tetrapack','surgele,carton','barquette,film,plastique,frais,sous-atmosphere-protectrice','sous-atmosphere-protectrice',
                                  'carton,plastique,frais','frais,pot,plastique'],'Non-Recyclable'))


data['emballage'] = data['packaging_tags'].replace(pack_mapping)

In [21]:
data['emballage'].value_counts()

Recyclable                                           46498
Plastic                                              45494
Non-Recyclable                                        6570
sachet                                                3352
barquette                                             1511
                                                     ...  
putignano-ba                                             1
plastic-film,glass-jar,metal-lid,carton-board,pot        1
frais,pot-et-opercule-a-jeter                            1
pot,verre,couvercle,capsule                              1
carton,sachet-protecteur                                 1
Name: emballage, Length: 26508, dtype: int64

# NUTRISCORE
https://quoidansmonassiette.fr/comment-est-calcule-le-nutri-score-logo-nutritionnel/



In [33]:
# energy
energy_bins = [data['energy_100g'].min() - 1, 335, 670, 1005, 1340, 1675, 2010, 2345, 2680, 3015, 3350, data['energy_100g'].max()]
data['energy_points'] = pd.cut(data['energy_100g'], energy_bins, labels=range(11)).astype(float)


# sugar aliments
sugar_aliments_bins = [data['sugars_100g'].min() - 1, 0, 1.5, 3, 4.5, 6, 7.5, 9, 10.5, 12, 13.5, data['sugars_100g'].max()]
data['sugar_points'] = pd.cut(data['sugars_100g'], sugar_bins, labels=range(11)).astype(float)
# sugar beverages
sugar_beverages_bins = [data['sugars_100g'].min() - 1, 4.5, 9, 13.5, 18, 22.5, 27, 31, 36, 40, 45, data['sugars_100g'].max()]
data['sugar_points'] = pd.cut(data['sugars_100g'], sugar_bins, labels=range(11)).astype(float)



# fat aliments
fat_bins = [data['saturated-fat_100g'].min() - 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, data['saturated-fat_100g'].max()]
data['saturated-fat_points'] = pd.cut(data['saturated-fat_100g'], fat_bins, labels=range(11)).astype(float)
# fat fat sauces
fat_fatsauces_bins = [data['saturated-fat_100g'].min() - 1, 10, 16, 22, 28, 34, 40, 46, 52, 58, 64, data['saturated-fat_100g'].max()]
data['saturated-fat_points'] = pd.cut(data['saturated-fat_100g'], fat_bins, labels=range(11)).astype(float)


# sodium
sodium_bins = [data['sodium_100g'].min() - 1, 90, 180, 270, 360, 450, 540, 630, 720, 810, 900, data['sodium_100g'].max()]
data['sodium_points'] = pd.cut(data['sodium_100g'], sodium_bins, labels=range(11)).astype(float)



In [ ]:
data['points_N'] = data['energy_points'] + data['fat_points'] + data['sugar_points'] + data['sodium_points']

In [36]:
data[data['Category_Food']=='Fat Sauces']['saturated-fat_100g'].value_counts()

0.000     4047
0.100     1107
14.000     770
13.330     706
13.000     636
          ... 
0.890        1
67.900       1
20.200       1
85.200       1
9.209        1
Name: saturated-fat_100g, Length: 904, dtype: int64

0          4.0
1          NaN
2          2.0
3          NaN
4          0.0
          ... 
1038901    4.0
1038902    4.0
1038903    1.0
1038904    NaN
1038905    NaN
Name: energy_points, Length: 1038906, dtype: float64